In [1]:
import pyomo.environ as pyo


In [2]:

model: pyo.AbstractModel = pyo.AbstractModel() #  type: ignore

model.I = pyo.Set()

model.L= pyo.Set()
model.S = pyo.Set(within=model.L)
model.F= pyo.Set(model.L, within=model.I *model.I)
model.LF = pyo.Set(
    dimen=3, 
    initialize=lambda model: [(l, f) for l in model.L for f in model.F[l]]
)

model.nS = pyo.Set(
    dimen=1, 
    initialize=lambda model: [l for l in model.L if l not in model.S]
)

model.X = pyo.Param(model.L, default=0.0, mutable=True)
model.Z = pyo.Param(model.I, default=20.0)
model.P = pyo.Var(model.LF, domain=pyo.NonNegativeReals)

def test_propagation(model, l):
        left = model.F[l].first()
        right = model.F[l].last()
        return model.P[l, left] + model.P[l, right] == model.X[l]
    

def test_objective(model):
    return 10

In [3]:
data = {
        None: {
            'I': {None: list(range(4))},
            "L": {None: list(range(3))},
            "F": {0: [(1, 0), (0, 1)], 1: [(2, 1), (1, 2)], 2: [(3, 2), (2, 3)]},
            "S": {None: list(range(1))},
            "X": {
                0: 10,
                1: 35,
                2: 25,
            },
            'Z': {0 : 100.0, 1: 50.0, 2: 200.0}
        }
    }

In [4]:
model.objective = pyo.Objective(rule=test_objective, sense=pyo.minimize)
model.test_propagation = pyo.Constraint(model.L, rule=test_propagation)



instance: pyo.Model = model.create_instance(data)

solver = pyo.SolverFactory('gurobi')

solver.solve(instance)

{'Problem': [{'Name': 'x1', 'Lower bound': 10.0, 'Upper bound': 10.0, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 7, 'Number of binary variables': 0, 'Number of integer variables': 0, 'Number of continuous variables': 7, 'Number of nonzeros': 6, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': 0, 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': 0.0010921955108642578, 'Error rc': 0}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [5]:
print(instance.P.extract_values()) # type: ignore
instance.X.store_values({0 : 100.0, 1: 50.0, 2: 200.0})
solver.solve(instance)
print(instance.P.extract_values()) # type: ignore

{(0, 1, 0): 10.0, (0, 0, 1): 0.0, (1, 2, 1): 35.0, (1, 1, 2): 0.0, (2, 3, 2): 25.0, (2, 2, 3): 0.0}
{(0, 1, 0): 100.0, (0, 0, 1): 0.0, (1, 2, 1): 50.0, (1, 1, 2): 0.0, (2, 3, 2): 200.0, (2, 2, 3): 0.0}


In [14]:
instance.objective.pprint()
bender_cut = 10*instance.P[0, 1, 0]

instance.objective.set_value(expr=instance.objective.expr + bender_cut)
instance.objective.pprint()

solver.solve(instance)
print(instance.P.extract_values()) 

objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 10.0 + 10*P[0,1,0]
objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 10.0 + 10*P[0,1,0] + 10*P[0,1,0]
{(0, 1, 0): 0.0, (0, 0, 1): 100.0, (1, 2, 1): 50.0, (1, 1, 2): 0.0, (2, 3, 2): 200.0, (2, 2, 3): 0.0}
